In [18]:
import sqlite3

import pandas as pd
from IPython.display import display

## Connect to SQLite database


In [19]:
# Define SQLite database file path
sqlite_db_path = "../data/vaccination_db.sqlite"

# Reconnect to SQLite database
conn = sqlite3.connect(sqlite_db_path)

## Query Database

To query the database, define the `query`. 

Next, execute the query and load the results into a DataFrame: `pd.read_sql_query(query, conn)`

### check info for all tables

In [20]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print(f"There are {len(tables)} tables in this sqlite dataset")
for table in tables:
    table_name = table[0]
    print(f"\nName of tables: {table_name}")

    # count rows for the table
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    row_count = cursor.fetchone()[0]
    print(f"row counts: {row_count}")
    # df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    # print(df.describe())

There are 6 tables in this sqlite dataset

Name of tables: Users
row counts: 200

Name of tables: Vaccines
row counts: 8

Name of tables: Polyclinics
row counts: 30

Name of tables: BookingSlots
row counts: 750

Name of tables: VaccineRecords
row counts: 500

Name of tables: VaccineStockInventory
row counts: 201


#### User

In [21]:
# Query the Vaccines table
query = "SELECT * FROM Users"
# Execute the query and load results into a DataFrame
user_df = pd.read_sql(query, conn)
# Display the results
display(user_df)

,id,name,email,date_of_birth,gender
0,1,Alicia Phillips,richard15@example.org,1968-10-07 00:00:00,Male
1,2,Clayton Duarte,vanessa30@example.org,1960-08-15 00:00:00,Male
2,3,Cassandra Young,laurenhill@example.com,1946-09-12 00:00:00,Male
3,4,Scott Hill,bgriffin@example.com,1946-05-24 00:00:00,Male
4,5,Katie Davis,nnielsen@example.org,1990-11-29 00:00:00,Female
...,...,...,...,...,...
195,196,Margaret Gould,bryan12@example.net,1988-03-20 00:00:00,Male
196,197,Anthony Brown,michaelolsen@example.org,1973-09-18 00:00:00,Male
197,198,Julie Charles,andreacrawford@example.net,1951-06-26 00:00:00,Female
198,199,Ryan Thomas,reynoldschristopher@example.com,1985-10-05 00:00:00,Male


In [22]:
start_date = "1920-01-01"
end_date = "2025-03-01"

user_df["date_of_birth"] = pd.to_datetime(user_df["date_of_birth"])
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

is_in_range = (user_df["date_of_birth"] >= start_date) & (
    user_df["date_of_birth"] <= end_date
)

# check if the date is under range
if is_in_range.all():
    print("All date are in range")
else:
    print("Some dates are not in range")

All date are in range


#### Vaccines

In [23]:
# Query the Vaccines table
query = "SELECT * FROM Vaccines"
# Execute the query and load results into a DataFrame
vaccines_df = pd.read_sql(query, conn)
# Display the results
display(vaccines_df)

,id,vaccine,price,doses_required,age_criteria,gender_criteria
0,1,Influenza (INF),9.0,2,18+ years old,None
1,2,Human Papillomavirus (HPV),0.0,4,18+ years old,None
2,3,Pneumococcal,16.0,1,65+ years old,None
3,4,Human Immunodeficiency Viruses (HIV),23.0,2,18-26 years old,Female
4,5,"Tetanus, Diphtheria, Pertussis (Tdap)",10.0,1,27-64 years old,Female
5,6,Hepatitis B (HepB),9.0,2,18+ years old,None
6,7,"Measles, Mumps, Rubella (MMR)",9.0,1,18+ years old,None
7,8,Varicella (VAR),11.0,3,18+ years old,None


#### Polyclinics

In [24]:
# Query the Vaccines table
query = "SELECT * FROM Polyclinics"
# Execute the query and load results into a DataFrame
polyclinic_df = pd.read_sql(query, conn)
# Display the results
display(polyclinic_df)

,id,name,address
0,1,Riley Ltd Polyclinic,95509 Chavez Mission Apt. 305\nEast Matthewsid...
1,2,Williams-Lee Polyclinic,"10208 Jennifer Underpass\nEast Raymond, AZ 45168"
2,3,"Phillips, Larsen and Anderson Polyclinic","9210 Davis Wall\nKrystalport, VT 22598"
3,4,Dorsey-Ruiz Polyclinic,"959 Eaton Loaf\nWest Jennashire, AK 83915"
4,5,Cohen Ltd Polyclinic,"1724 Cowan Throughway\nGeorgeport, PW 46408"
5,6,Sawyer LLC Polyclinic,"241 Brown Village\nMarkhaven, NM 53988"
6,7,Anderson and Sons Polyclinic,"13457 William Garden\nLake Kristen, AR 59309"
7,8,Odom Inc Polyclinic,"943 Avery Forks Suite 506\nPort Brenda, MO 61245"
8,9,"Grant, Johnson and Thomas Polyclinic","360 Jimenez Knolls Suite 324\nJamesland, PW 15865"
9,10,Barker LLC Polyclinic,"483 Lisa Shoal\nGinafort, MA 92919"


#### BookingSlots

In [25]:
# Query the Vaccines table
query = "SELECT * FROM BookingSlots"
# Execute the query and load results into a DataFrame
bookingslot_df = pd.read_sql(query, conn)
# Display the results
display(bookingslot_df)

id polyclinic_id vaccine_id        slot_datetime  is_booked user_id
0      1            29          1  2025-03-20 15:00:00          0    None
1      2            21          7  2025-03-19 11:00:00          1      51
2      3            14          2  2025-03-18 16:00:00          1      72
3      4            24          7  2025-03-18 18:00:00          1     107
4      5            21          7  2025-03-17 11:00:00          1      82
..   ...           ...        ...                  ...        ...     ...
745  746             3          8  2025-03-19 18:00:00          0    None
746  747            14          7  2025-03-17 11:00:00          0    None
747  748            27          1  2025-03-17 16:00:00          1      26
748  749            18          1  2025-03-21 13:00:00          1     103
749  750            18          5  2025-03-20 16:00:00          0    None

[750 rows x 6 columns]

In [26]:
bookingslot_df["slot_datetime"].value_counts()

slot_datetime
2025-03-18 09:00:00    22
2025-03-19 11:00:00    21
2025-03-20 11:00:00    21
2025-03-21 12:00:00    21
2025-03-21 15:00:00    20
2025-03-17 17:00:00    20
2025-03-21 18:00:00    19
2025-03-18 18:00:00    19
2025-03-17 11:00:00    19
2025-03-17 16:00:00    19
2025-03-20 18:00:00    19
2025-03-21 13:00:00    18
2025-03-21 10:00:00    18
2025-03-20 15:00:00    18
2025-03-17 10:00:00    18
2025-03-21 09:00:00    18
2025-03-18 16:00:00    17
2025-03-20 16:00:00    17
2025-03-20 10:00:00    17
2025-03-19 09:00:00    17
2025-03-19 12:00:00    17
2025-03-19 14:00:00    16
2025-03-20 17:00:00    15
2025-03-17 09:00:00    15
2025-03-21 14:00:00    15
2025-03-20 12:00:00    15
2025-03-21 17:00:00    15
2025-03-18 13:00:00    15
2025-03-17 12:00:00    14
2025-03-19 13:00:00    14
2025-03-17 15:00:00    14
2025-03-18 12:00:00    14
2025-03-19 16:00:00    13
2025-03-19 15:00:00    13
2025-03-20 09:00:00    13
2025-03-18 11:00:00    13
2025-03-19 17:00:00    13
2025-03-17 14:00:00    1

#### VaccineRecords

In [27]:
# Query the Vaccines table
query = "SELECT * FROM VaccineRecords"
# Execute the query and load results into a DataFrame
vaccinerecord_df = pd.read_sql(query, conn)
# Display the results
display(vaccinerecord_df)

,id,user_id,vaccine_id,polyclinic_id,vaccination_date
0,1,4,6,16,2025-02-16
1,2,32,6,25,2025-02-20
2,3,102,5,14,2025-02-16
3,4,173,2,12,2025-01-08
4,5,99,2,17,2025-01-29
...,...,...,...,...,...
495,496,1,7,19,2025-03-07
496,497,136,8,10,2025-02-21
497,498,183,7,24,2025-02-27
498,499,45,2,18,2025-02-05


In [28]:
vaccinerecord_df["vaccination_date"].sort_values()

357    2025-01-01
128    2025-01-01
74     2025-01-01
187    2025-01-01
62     2025-01-01
          ...    
489    2025-03-19
56     2025-03-19
483    2025-03-19
417    2025-03-19
464    2025-03-19
Name: vaccination_date, Length: 500, dtype: object

#### VaccineStockInventory

In [29]:
# Query the Vaccines table
query = "SELECT * FROM VaccineStockInventory"
# Execute the query and load results into a DataFrame
vaccinestock_df = pd.read_sql(query, conn)
# Display the results
display(vaccinestock_df)

id polyclinic_id vaccine_id  stock_quantity
0      1            26          5              52
1      2            26          4             167
2      3            26          1              90
3      4            26          6             158
4      5            26          3             119
..   ...           ...        ...             ...
196  197            30          8             129
197  198            16          7              90
198  199            13          1             142
199  200            13          4              70
200  201            13          6              63

[201 rows x 4 columns]

In [13]:
# Group the dataframe by polyclinic_id and vaccine_id and count occurrences
grouped = (
    vaccinestock_df.groupby(["polyclinic_id", "vaccine_id"])
    .size()
    .reset_index(name="count")
)

# Check if any combination appears more than once
duplicates = grouped[grouped["count"] > 1]

# Display the duplicates if any
if not duplicates.empty:
    print("Found duplicate entries:")
    display(duplicates)
else:
    print("All polyclinic_id and vaccine_id combinations are unique.")

All polyclinic_id and vaccine_id combinations are unique.


## Close SQLite Database Connection


In [30]:
# Close the connection
conn.close()